# Projeto 1 - Ciência dos Dados

Nome: 

Nome: 

    Nome: Mateus Ruggero SUAVIZAÇÃO DE LAPLACE NÃO ESQUECE
    

Atenção: Serão permitidos grupos de três pessoas, mas com uma rubrica mais exigente. Grupos deste tamanho precisarão fazer um questionário de avaliação de trabalho em equipe

___
Carregando algumas bibliotecas:

In [16]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os

In [17]:
print('Esperamos trabalhar no diretório')
print(os.getcwd())

Esperamos trabalhar no diretório
C:\Users\joaop\Desktop\insper 2\ciencia dos dados\Projeto_1\Pojeto1_c_dados


Carregando a base de dados com os tweets classificados como relevantes e não relevantes:

In [18]:
filename = 'Shang Chi.xlsx'

In [19]:
train = pd.read_excel(filename)
train.head(5)

,Treinamento,Relevancia
0,maldito capitalismo que não me deixa ir ver sh...,3
1,💭💭 shang chi usando fones de ouvido 💭💭,0
2,queria assistir shang chi,3
3,uma blusinha de shang chi pf renner riachuelo ...,1
4,indo assistir shang chi sozinha espero que ngn...,3


In [20]:
test = pd.read_excel(filename, sheet_name = 'Teste')
test.head(5)

,Teste,Relevancia
0,esperando um drive de shang chi decente,1
1,tô falando: vão ver shang chi 🤩 https://t.co/u...,1
2,assisti shang - chi e pra mim é o melhor filme...,2
3,sabe quando vc quer fazer uma coisa q sabe q e...,1
4,n tem uma mísera alma pra ir ver shang chi com...,1


___
## Classificador automático de sentimento


Faça aqui uma descrição do seu produto e o que considerou como relevante ou não relevante na classificação dos tweets.

O produto escolhido foi o filme Shang-Chi e a lenda dos dez anéis.
Para avaliar como relevante, bastava o usuario ter visto o filme e mostrado interesse ou querer ver de novo, mas para ser muito relevante era preciso passar alguma crítica ao filme ou mostrar muito entusiasmo a respeito.

___
### Montando um Classificador Naive-Bayes

Considerando apenas as mensagens da planilha Treinamento, ensine  seu classificador.

In [21]:
import re 


def cleanup(text):
    """
        Função de limpeza muito simples que troca alguns sinais básicos por espaços
    """
    #import string
    punctuation = '[!-.:?;]' # Note que os sinais [] são delimitadores de um conjunto.
    pattern = re.compile(punctuation)
    text_subbed = re.sub(pattern, '', text)
    return text_subbed

In [22]:
###Transformando o total das palavras em DataFrame##############################################################################

total_raw = [] #Colocando frases em uma lista
for i in range(0, len(train)):
    total_raw.append(train['Treinamento'][i])

frases_lower = [] #Colocando as frases em lower case
for i in range(0, len(total_raw)):
    frases_lower.append(cleanup(total_raw[i].lower()))

total = [] #Transformando a lista em um índice para cada palavra
for frase in frases_lower:
    x = frase.split()
    for i in range(0, len(x)):
        total.append(x[i])

#Data Frame
total_serie = pd.Series(total)
total_relativo = total_serie.value_counts(True)

###Transformando os tweets irrelevantes em DataFrame############################################################################

irr_raw = [] #Colocando em uma lista
tweets_irrelevantes = train.loc[train['Relevancia'] == 0, :]

tweets_irrelevantes
for tweet in tweets_irrelevantes['Treinamento']:
    irr_raw.append(tweet)

irr_lower = [] #Colocando as frases em lower case
for i in range(0, len(irr_raw)):
    irr_lower.append(cleanup(irr_raw[i].lower()))

irr = [] #Transformando a lista em um índice para cada palavra
for frase in irr_lower:
    x = frase.split()
    for i in range(0, len(x)):
        irr.append(x[i])

#Data Frame
irr_serie = pd.Series(irr)
irr_relativo = irr_serie.value_counts(True)

# ###Transformando os tweets relevantes em DataFrame############################################################################

rel_raw = [] #Colocando em uma lista
tweets_relevantes = train.loc[train['Relevancia'] == 1, :]

for tweet in tweets_relevantes['Treinamento']:
    rel_raw.append(tweet)

rel_lower = [] #Colocando as frases em lower case
for i in range(0, len(rel_raw)):
    rel_lower.append(cleanup(rel_raw[i].lower()))

rel = [] #Transformando a lista em um índice para cada palavra
for frase in rel_lower:
    x = frase.split()
    for i in range(0, len(x)):
        rel.append(x[i])

#Data Frame
rel_serie = pd.Series(rel)
rel_relativo = rel_serie.value_counts(True)

###Transformando os tweets muito relevantes em DataFrame########################################################################

mt_rel_raw = [] #Colocando em uma lista
tweets_muito_relevantes = train.loc[train['Relevancia'] == 2, :]

for tweet in tweets_muito_relevantes['Treinamento']:
    mt_rel_raw.append(tweet)

mt_rel_lower = [] #Colocando as frases em lower case
for i in range(0, len(mt_rel_raw)):
    mt_rel_lower.append(cleanup(mt_rel_raw[i].lower()))

mt_rel = [] #Transformando a lista em um índice para cada palavra
for frase in mt_rel_lower:
    x = frase.split()
    for i in range(0, len(x)):
        mt_rel.append(x[i])

#Data Frame
mt_rel_serie = pd.Series(mt_rel)
mt_rel_relativo = mt_rel_serie.value_counts(True)

###Transformando os tweets muito relevantes em DataFrame########################################################################

int_raw = [] #Colocando em uma lista
tweets_int = train.loc[train['Relevancia'] == 3, :]

for tweet in tweets_int['Treinamento']:
    int_raw.append(tweet)

int_lower = [] #Colocando as frases em lower case
for i in range(0, len(int_raw)):
    int_lower.append(cleanup(int_raw[i].lower()))

int_ver = [] #Transformando a lista em um índice para cada palavra
for frase in int_lower:
    x = frase.split()
    for i in range(0, len(x)):
        int_ver.append(x[i])

#Data Frame
int_serie = pd.Series(mt_rel)
int_relativo = int_serie.value_counts(True)

In [23]:
###Cálculo de P(I), P(R) e P(MR)################################################################################################

probI = len(irr_serie)/len(total_serie)
probR = len(rel_serie)/len(total_serie)
probMR = len(mt_rel_serie)/len(total_serie)
probIV = len(int_serie)/len(total_serie)

###Cálculo da probabilidade#####################################################################################################

def Prob(tweet):
    twt = cleanup(tweet.lower()) #Limpar o tweet
    tweet_certo = twt.split()
    
    ProbTweetDadoI = 1
    ProbTweetDadoR = 1
    ProbTweetDadoMR = 1
    ProbTweetDadoIV = 1
    
  #Cálculo de P(tweet|I), P(tweet|R) e P(tweet|MR)##############################################################################
    
    for palavra in tweet_certo:
        
        if palavra in irr_serie.value_counts():
            cont_palavra_irr = irr_serie.value_counts()[palavra]
        else:
            cont_palavra_irr = 0
            
        if palavra in rel_serie.value_counts():
            cont_palavra_rel = rel_serie.value_counts()[palavra]
        else:
            cont_palavra_rel = 0
            
        if palavra in mt_rel_serie.value_counts():
            cont_palavra_mt_rel = mt_rel_serie.value_counts()[palavra]
        else:
            cont_palavra_mt_rel = 0
            
        if palavra in int_serie.value_counts():
            cont_palavra_int = int_serie.value_counts()[palavra]
        else:
            cont_palavra_int = 0
    
        ProbTweetDadoI = ProbTweetDadoI * ((cont_palavra_irr + 1)/(len(irr_serie) + len(total_relativo)))
        ProbTweetDadoR = ProbTweetDadoR * ((cont_palavra_rel + 1)/(len(rel_serie) + len(total_relativo)))
        ProbTweetDadoMR = ProbTweetDadoMR * ((cont_palavra_mt_rel + 1)/(len(mt_rel_serie) + len(total_relativo)))
        ProbTweetDadoIV = ProbTweetDadoIV * ((cont_palavra_int + 1)/(len(int_serie) + len(total_relativo)))

  #Cálculo de P(I|tweet), P(R|tweet) e P(MR|tweet)##############################################################################
    P_I_tweet = ProbTweetDadoI * probI
    P_R_tweet = ProbTweetDadoR * probR
    P_MR_tweet = ProbTweetDadoMR * probMR
    P_I_V_tweet = ProbTweetDadoIV * probIV
    
    lista = [P_I_tweet,P_R_tweet,P_MR_tweet,]
    
    if P_I_tweet == max(lista):
        return 0
    elif P_R_tweet == max(lista):
        return 1
    elif P_MR_tweet == max(lista):
        return 2
    else:
        return 3

___
### Verificando a performance do Classificador

Agora você deve testar o seu classificador com a base de Testes.

In [24]:
i = 0
lista_relevancia = []
for tweet in test['Teste']:
    y = Prob(tweet)
    lista_relevancia.append(y)
    i += 1
    
lista_relevancia

[1,
 1,
 2,
 1,
 1,
 1,
 1,
 1,
 2,
 1,
 1,
 0,
 1,
 2,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 2,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 2,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 2,
 1,
 1,
 1,
 1,
 2,
 2,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 2,
 2,
 2,
 2,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 2,
 2,
 1,
 2,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 2,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 2,
 0,
 1,
 1,
 1,
 1,
 1,
 2,
 1,
 1,
 2,
 2,
 2,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 2,
 1,
 1,
 2,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 2,
 2,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 2,
 1,
 2,
 1,
 0,
 2,
 1,
 1,
 1,
 1,
 1,
 2,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 2,
 0,
 1,
 1]

In [25]:
array = np.array(lista_relevancia)
pd.crosstab(test['Relevancia'], array, margins = True, normalize = 'index')

col_0,0,1,2
Relevancia,,,
0,0.327273,0.563636,0.109091
1,0.077922,0.883117,0.038961
2,0.161290,0.161290,0.677419
3,0.027027,0.972973,0.000000
All,0.150000,0.700000,0.150000


In [26]:
train['Relevancia'].value_counts()

1    186
3    133
0    127
2    104
Name: Relevancia, dtype: int64

In [27]:
test['Relevancia'].value_counts()

1    77
0    55
3    37
2    31
Name: Relevancia, dtype: int64

___
### Concluindo

In [28]:
II = 17/56 * 100
IO = (56-17)/56 * 100
RR = 61/71 * 100
RO =(71-61)/71 * 100
MRMR = 20/33 * 100
MRO = (33-20)/33 * 100
IVIV = 0
IVO = 100
CC = (17+61+20)/200 * 100
print(II)
print(IO)
print(RR)
print(RO)
print(MRMR)
print(MRO)
print(IVIV)
print(IVO)
print(CC)

30.357142857142854
69.64285714285714
85.91549295774648
14.084507042253522
60.60606060606061
39.39393939393939
0
100
49.0


___
### Qualidade do Classificador a partir de novas separações dos tweets entre Treinamento e Teste

Caso for fazer esse item do Projeto

___
## Aperfeiçoamento:

Trabalhos que conseguirem pelo menos conceito B vão evoluir em conceito dependendo da quantidade de itens avançados:

* IMPLEMENTOU outras limpezas e transformações que não afetem a qualidade da informação contida nos tweets. Ex: stemming, lemmatization, stopwords
* CORRIGIU separação de espaços entre palavras e emojis ou entre emojis e emojis
* CRIOU categorias intermediárias de relevância baseadas na probabilidade: ex.: muito relevante, relevante, neutro, irrelevante, muito irrelevante. Pelo menos quatro categorias, com adição de mais tweets na base, conforme enunciado. (OBRIGATÓRIO PARA TRIOS, sem contar como item avançado)
* EXPLICOU porquê não pode usar o próprio classificador para gerar mais amostras de treinamento
* PROPÔS diferentes cenários para Naïve Bayes fora do contexto do projeto
* SUGERIU e EXPLICOU melhorias reais com indicações concretas de como implementar (indicar como fazer e indicar material de pesquisa)
* FEZ o item 6. Qualidade do Classificador a partir de novas separações dos tweets entre Treinamento e Teste descrito no enunciado do projeto (OBRIGATÓRIO para conceitos A ou A+)

___
## Referências

[Naive Bayes and Text Classification](https://arxiv.org/pdf/1410.5329.pdf)  **Mais completo**

[A practical explanation of a Naive Bayes Classifier](https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/) **Mais simples**